<a href="https://colab.research.google.com/github/gcsilva83/ChatGPT_Notebooks/blob/main/Text_To_SQL_alpaca_native.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install deepspeed
!pip install accelerate
!pip install sentencepiece
!pip install langchain
!pip install torch
!pip install bitsandbytes
!pip install scipy
!pip install torch
!pip install xformers
!pip install transformers

In [4]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [ ]:
base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
)

In [8]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

In [22]:
from langchain import PromptTemplate, LLMChain

template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question :
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

In [23]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

In [24]:
def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt,
                         llm=local_llm
                         )
    response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    print(response)

In [25]:
tble = "employee"
cols = ["id","name","date_of_birth","band","manager_id"]
question = "Query the count of employees in band L6 with 239045 as the manager ID"
output = get_llm_response(tble,question,cols)

Answer: SELECT COUNT(*) FROM employee WHERE band='L6' AND manager_ID=239045;


In [26]:
tble = ["sales","product","customer"]
cols = ["id_sales","name_sales","order_date","order_product","product_order_name", "total_EUR"]
question = "Query the total amount Total_EUR of product in the last 30 days by order_date"
output = get_llm_response(tble,question,cols)

Answer: SELECT SUM(Total_EUR) FROM sales WHERE OrderDate >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);


In [30]:

tble = ["sales","product","customer"]
cols = ["id_sales","name_sales","order_date","order_product","product_order_name", "total_EUR"]
question = "Query the total amount Total_EUR of product in the last 30 days by order_date per day and product ordered"
output = get_llm_response(tble,question,cols)

Answer:
SELECT SUM(Total_EUR) AS Total_EUR, COUNT(*) AS Ordered_Product
FROM sales s INNER JOIN customer c ON s.order_product = c.ID
WHERE s.order_date > CURDATE() - INTERVAL 30 DAY;
